### Proximal Policy Optimization

**Principle:** Constrained Policy Optimization

**Definition:** Maximizes surrogate objective while clipping probability ratios to ensure stable updates (PPO).

**Algorithm Description:** Proximal Policy Optimization clips policy updates to prevent excessively large changes, ensuring new policies don't deviate too far from old ones. This clipping mechanism provides a simpler alternative to trust region methods while maintaining stable, reliable learning across various environments and action spaces.

**Typical Use Cases:**
- Balances sample efficiency and ease of tuning.
- Robust, general-purpose policy gradient algorithm
- Simpler to implement than trpo
- When stability and reliable performance are needed
- Works with continuous and discrete action spaces

**Assumptions:**
- Continuous/discrete actions
- Large datasets
- On-policy learning
- Stable updates



### ⚙️ Prerequisites & Setup

**Required Packages:**
- `gymnasium>=0.29.0` - RL environment
- `torch>=2.0.0` - Deep learning framework
- `numpy`, `matplotlib`, `seaborn` - Data processing and visualization

**Installation:**
```bash
pip install gymnasium torch numpy matplotlib seaborn
```

**⏱️ Estimated Training Time:**
- CPU: ~10-15 minutes
- GPU: ~4-6 minutes

**💡 Quick Test:** To reduce training time, change `n_episodes=500` to `n_episodes=100` in the training cell.

### 1. Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
torch.manual_seed(42)

### 2. PPO Algorithm

PPO (Proximal Policy Optimization) uses a clipped surrogate objective:

**L^CLIP(θ) = E[min(r_t(θ)A_t, clip(r_t(θ), 1-ε, 1+ε)A_t)]**

Where:
- r_t(θ) = π_θ(a|s) / π_θ_old(a|s) (probability ratio)
- A_t = advantage estimate
- ε = clip parameter (typically 0.2)

**Key advantage:** Prevents large policy updates

In [ ]:
class ActorCriticNetwork(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=64):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU()
        )
        self.actor = nn.Linear(hidden_dim, action_dim)
        self.critic = nn.Linear(hidden_dim, 1)
    
    def forward(self, state):
        x = self.shared(state)
        return F.softmax(self.actor(x), dim=-1), self.critic(x)

In [ ]:
class PPOAgent:
    def __init__(self, state_dim, action_dim, lr=3e-4, gamma=0.99, 
                 eps_clip=0.2, K_epochs=4):
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        
        self.policy = ActorCriticNetwork(state_dim, action_dim)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=lr)
        self.policy_old = ActorCriticNetwork(state_dim, action_dim)
        self.policy_old.load_state_dict(self.policy.state_dict())
        
        self.buffer = {'states': [], 'actions': [], 'rewards': [], 
                      'log_probs': [], 'dones': []}
    
    def select_action(self, state):
        with torch.no_grad():
            state = torch.FloatTensor(state)
            probs, _ = self.policy_old(state)
            dist = Categorical(probs)
            action = dist.sample()
            log_prob = dist.log_prob(action)
        
        self.buffer['states'].append(state)
        self.buffer['actions'].append(action)
        self.buffer['log_probs'].append(log_prob)
        return action.item()
    
    def update(self):
        # Calculate returns
        returns = []
        G = 0
        for r, done in zip(reversed(self.buffer['rewards']), 
                          reversed(self.buffer['dones'])):
            if done:
                G = 0
            G = r + self.gamma * G
            returns.insert(0, G)
        
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + 1e-7)
        
        old_states = torch.stack(self.buffer['states'])
        old_actions = torch.stack(self.buffer['actions'])
        old_log_probs = torch.stack(self.buffer['log_probs'])
        
        # PPO update for K epochs
        for _ in range(self.K_epochs):
            probs, state_values = self.policy(old_states)
            dist = Categorical(probs)
            log_probs = dist.log_prob(old_actions)
            entropy = dist.entropy()
            
            # Calculate ratio and clipped objective
            ratios = torch.exp(log_probs - old_log_probs.detach())
            advantages = returns - state_values.squeeze().detach()
            
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages
            
            loss = -torch.min(surr1, surr2).mean() + \
                   0.5 * F.mse_loss(state_values.squeeze(), returns) - \
                   0.01 * entropy.mean()
            
            self.optimizer.zero_grad()
            loss.backward()
            
            # Gradient clipping for training stability
            torch.nn.utils.clip_grad_norm_(self.policy.parameters(), max_norm=0.5)
            
            self.optimizer.step()
        
        # Update old policy
        self.policy_old.load_state_dict(self.policy.state_dict())
        
        # Clear buffer
        self.buffer = {'states': [], 'actions': [], 'rewards': [], 
                      'log_probs': [], 'dones': []}

### 3. Train PPO Agent

In [ ]:
env = gym.make('CartPole-v1')
agent = PPOAgent(env.observation_space.shape[0], env.action_space.n)

episode_rewards = []
update_interval = 20

print('Training PPO...')
for episode in range(500):
    state, _ = env.reset()
    total_reward = 0
    
    for t in range(500):
        action = agent.select_action(state)
        state, reward, terminated, truncated, _ = env.step(action)
        agent.buffer['rewards'].append(reward)
        agent.buffer['dones'].append(terminated or truncated)
        total_reward += reward
        
        if terminated or truncated:
            break
    
    episode_rewards.append(total_reward)
    
    if (episode + 1) % update_interval == 0:
        agent.update()
    
    if (episode + 1) % 50 == 0:
        print(f'Episode {episode+1}, Avg Reward: {np.mean(episode_rewards[-50:]):.2f}')

### 4. Visualize Results

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(episode_rewards, alpha=0.3)
if len(episode_rewards) > 20:
    ma = np.convolve(episode_rewards, np.ones(20)/20, mode='valid')
    plt.plot(ma, linewidth=2, label='MA(20)')
plt.axhline(195, color='r', linestyle='--', label='Solved')
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.title('PPO Training Performance')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

print(f'Final performance: {np.mean(episode_rewards[-100:]):.2f}')
env.close()